# Introduction to Neural Networks: Logistic Regression and Loss

Author: Pierre Nugues

Initial dataset from Joseph Berkson on *Application of the Logistic Function to Bio-Assay* (1944), (https://www.jstor.org/stable/2280041)

In [ ]:
dose = [40, 60, 80, 100, 120, 140, 160, 180, 200, 250, 300]
exposed = [462, 500, 467, 515, 561, 469, 550, 542, 479, 497, 453]
mortality = [109, 199, 298, 370, 459, 400, 495, 499, 450, 476, 442]
dataset = list(zip(dose, exposed, mortality))
dataset

## Formatting the Data

Following Berkson, $y = 1$ is die, while $y = 0$ is survive

In [ ]:
from math import log10
import numpy as np

X = []
y = []
for obs in dataset:
    mortality_nbr = obs[2]
    survival_nbr = obs[1] - mortality_nbr
    for _ in range(mortality_nbr):
        X += [obs[0]]
        y += [1]
    for _ in range(survival_nbr):
        X += [obs[0]]
        y += [0]

#X = list(map(log10, X))
X = np.array(X).reshape(-1, 1)
y = np.array(y).reshape(-1, 1)
np.hstack((X, y))

## Using the Logistic Curve (Sigmoid)

A simple logictic regression architecture

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

np.random.seed(0)

model = Sequential([
    Dense(1, input_dim=1, activation='sigmoid')])
model.compile(optimizer='nadam', loss='binary_crossentropy', 
              metrics=['accuracy'])
model.summary()

We fit it

In [ ]:
history = model.fit(X, y, epochs=50, verbose=0)
model.evaluate(X, y)

## The Model Weights

In [ ]:
model.get_weights()

## The Curves

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
loss = history.history['loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'ro', label='Training acc')
plt.title('Training accuracy and loss')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.title('Training accuracy and loss')
plt.show()

## Computing the Loss

The crossentropy loss is defined as:
$
H(P,M) = - \frac{1}{|X|} \sum\limits_{x \in X} {P(x)\log M(x),} 
$
where $P$ and $M$ its probability.

### First, a simple example:
We draw individuals

In [ ]:
from math import log

random_observations = np.random.choice(X.shape[0], 6, replace = False)
print(random_observations)

Which dose did they ingest and what was the outcome?

In [ ]:
X_choice = list(X[random_observations])
print('Dose:', X_choice)
y_choice = list(y[random_observations])
print('Observed class:', y_choice)

What is the predicted outcome?

In [ ]:
probs_raw = list(map(model.predict, X_choice))
print('Predicted probs for class 1:', probs_raw)

Let us compute the cross-entropy

In [ ]:
probs = [log(p) * y + log(1 - p) * (1 - y) for p, y in zip(probs_raw, y_choice)]
print('Crossentropy per observation:', probs)
entropy = -sum(probs)/6
print('Crossentropy:', entropy)

### The whole dataset
We compute it over the whole training set

In [ ]:
probs_raw = model.predict(X)
probs = [log(p) * y + log(1 - p) * (1 - y) for p, y in zip(probs_raw, y)]
-sum(probs)/X.shape[0]